### Use ``Open Route Service API`` to retrieve distance matrix (Single Call version)
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 
#### The following provides a sample call for 2017

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2017]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude']).reset_index(drop=True)
print(place_df.shape)
place_df.head(10)

(8284, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,ALAMEIN PRIMARY SCHOOL,-37.86707,145.08042,2017,primary,207011146
1,NUNAWADING PRIMARY SCHOOL,-37.80747,145.16699,2017,primary,207031161
2,CRINIGAN ROAD PRIMARY SCHOOL,-38.21790,146.41737,2017,primary,205041096
3,SPRINGBANK PRIMARY SCHOOL,-37.53207,144.05265,2017,primary,201021012
4,WATTLE PARK PRIMARY SCHOOL,-37.84264,145.09737,2017,primary,207031165
5,OMEO PRIMARY SCHOOL,-37.10022,147.59308,2017,primary,205021082
6,Newcombe Park Primary School,-38.16515,144.39496,2017,primary,203021045
7,MEADOWBANK PRIMARY SCHOOL,-37.68790,144.92903,2017,primary,210051242
8,PLENTY PARKLANDS PRIMARY SCHOOL,-37.66972,145.07694,2017,primary,209041220
9,MOUNT PROSPECT PRIMARY SCHOOL,-37.37818,144.01848,2017,primary,201021011


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2017_property_with_SA2.csv')[PROPERTY_COLS]
# 2017 example
print(property_df.shape)
property_df.head(10)

(17143, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027
1,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2017,4,House,202021027
2,"22 PITMAN STREET, CHEWTON",-37.084464,144.266909,2.0,1.0,2,320,House,3451,2017,9,House,202021027
3,"22 PITMAN STREET, CHEWTON",-37.084464,144.266909,2.0,1.0,2,320,House,3451,2017,8,House,202021027
4,"14 PITMAN STREET, CHEWTON",-37.083013,144.266589,4.0,1.0,1,335,House,3451,2017,2,House,202021027
5,"239 MAIN ROAD, CHEWTON",-37.081393,144.266198,2.0,1.0,2,365,House,3451,2017,6,House,202021027
6,"52 FRYERS ROAD, CHEWTON",-37.087738,144.257555,3.0,1.0,2,450,House,3451,2017,8,House,202021027
7,"171 MAIN ROAD, CHEWTON",-37.081326,144.259245,3.0,1.0,8,335,House,3451,2017,3,House,202021027
8,"16 STEELE ST, CHEWTON",-37.079630,144.249108,3.0,2.0,2,350,House,3451,2017,12,House,202021027
9,"16 STEELE STREET, CHEWTON",-37.079729,144.248306,3.0,2.0,2,350,House,3451,2017,12,House,202021027


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(f"shape = {merged_df.shape}")
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

shape = (330166, 18)
Distinct sa2 codes = 491


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
1,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary
2,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,CASTLEMAINE PRIMARY SCHOOL,-37.06660,144.22233,primary
3,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,CHEWTON PRIMARY SCHOOL,-37.08233,144.26116,primary
4,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,CAMPBELLS CREEK PRIMARY SCHOOL,-37.09293,144.20466,primary
5,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,ST MARYS SCHOOL,-37.06396,144.22096,primary
6,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary
7,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,CASTLEMAINE TECHNICAL COLLEGE,-37.06346,144.22098,secondary
8,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,Castlemaine Secondary College- Junior Campus,-37.05054,144.22750,secondary
9,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,9,House,202021027,Castlemaine Secondary College - Senior Campus,-37.07707,144.21453,secondary


### Create the directory to store merged distance data

In [6]:
if not os.path.exists('../../data/featured'):
    os.makedirs('../../data/featured')

#### Specify the ``client`` and ``year`` here, as well as adding a mode of testing/saving

In [7]:
client = ors.Client('5b3ce3597851110001cf624851a53f74cc8e4c6c9e751bb8aee27e05') # hyunjinp api key 500

In [8]:
added_distance_merged_df = add_distance_time(merged_df, 2017, client, 'saving') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 202021027
Subset size = 630, Places count = 14, Property count = 45
To Place Distance Grand List, Normal Branch, length = 630
Executing SA2 Code 204031070
Subset size = 171, Places count = 9, Property count = 19
To Place Distance Grand List, Normal Branch, length = 801
Executing SA2 Code 210021235
Subset size = 210, Places count = 10, Property count = 21
To Place Distance Grand List, Normal Branch, length = 1011
Executing SA2 Code 211051275
Subset size = 27, Places count = 3, Property count = 9
To Place Distance Grand List, Normal Branch, length = 1038
Executing SA2 Code 209031212
Subset size = 312, Places count = 26, Property count = 12
To Place Distance Grand List, Normal Branch, length = 1350
Executing SA2 Code 211051276
Subset size = 792, Places count = 22, Property count = 36
To Place Distance Grand List, Normal Branch, length = 2142
Executing SA2 Code 205041094
Subset size = 1833, Places count = 39, Property count = 47
To Place Distance Grand List, Normal Branc

In [9]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.20749,primary,6769.73,6.76973,674.28,11.238000,118263.38,118.26338,5413.02,90.217000
1,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.21899,primary,6470.85,6.47085,478.77,7.979500,118263.38,118.26338,5413.02,90.217000
2,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.22233,primary,4956.06,4.95606,375.73,6.262167,118263.38,118.26338,5413.02,90.217000
3,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.26116,primary,1889.06,1.88906,175.94,2.932333,118263.38,118.26338,5413.02,90.217000
4,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.20466,primary,8837.92,8.83792,701.23,11.687167,118263.38,118.26338,5413.02,90.217000
5,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.22096,primary,5585.91,5.58591,492.38,8.206333,118263.38,118.26338,5413.02,90.217000
6,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.22750,secondary,8259.20,8.25920,656.65,10.944167,118263.38,118.26338,5413.02,90.217000
7,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.22098,secondary,5617.34,5.61734,496.87,8.281167,118263.38,118.26338,5413.02,90.217000
8,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.22750,secondary,8259.20,8.25920,656.65,10.944167,118263.38,118.26338,5413.02,90.217000
9,"4 ALBERT STREET, CHEWTON",-37.091814,144.258599,3.0,1.0,2,345,House,3451,2017,...,144.21453,secondary,6587.77,6.58777,680.03,11.333833,118263.38,118.26338,5413.02,90.217000
